<a href="https://colab.research.google.com/github/brusangues/maps-reviews-api-scraper/blob/reorganization/1_maps_reviews_api_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Dados Textuais

Utilizaremos uma ferramenta scraper para obter dados de texto de avaliações de hotéis no Google Maps.
Depois, faremos um tratamento básico desses dados.

## 1.1. Utilizando o Scraper para extração de Dados

In [1]:
!git clone --branch reorganization https://github.com/brusangues/maps-reviews-api-scraper

fatal: destination path 'maps-reviews-api-scraper' already exists and is not an empty directory.


In [4]:
%cd /content/maps-reviews-api-scraper/scraper

/content/maps-reviews-api-scraper/scraper


In [3]:
!pip install -r requirements.txt

In [6]:
%cd scraper/

c:\Users\Bruno\Documents\GitHub\pgc2_23_01_29\scraper


In [4]:
!ls

0_places.ipynb
1_maps_reviews_api_scraper.ipynb
2_large_language_models.ipynb
3_text_summarization.ipynb
4_chatbot.ipynb
LICENSE.txt
README.MD
__pycache__
_other
analysis
data
env
llms
paper
references.txt
reports
requirements.txt
scraper
sql
sql.ipynb
telegram_bot.py


In [7]:
!python -m app run-async --path ./input/test.csv

2025-02-12 13:36:38,109 INFO  18984 [    call_pools] (app.py: 46): - Running async
2025-02-12 13:36:40,040 INFO   1448 [  call_scraper] (app.py: 84): - folder created
2025-02-12 13:36:40,041 INFO  20188 [  call_scraper] (app.py: 84): - folder created
2025-02-12 13:36:40,057 INFO   1448 [  scrape_place] (scraper.py:479): Pousada Itararé  - Scraping metadata for url: Pousada Itararé
2025-02-12 13:36:40,057 INFO  20188 [  scrape_place] (scraper.py:479): Wish Foz do Igua - Scraping metadata for url: Wish Foz do Iguaçu
2025-02-12 13:36:40,058 INFO   1448 [  scrape_place] (scraper.py:483): Pousada Itararé  - Parsing metadata...
2025-02-12 13:36:40,058 INFO  20188 [  scrape_place] (scraper.py:483): Wish Foz do Igua - Parsing metadata...
2025-02-12 13:36:40,801 INFO   1448 [  call_scraper] (app.py:122): Pousada Itararé  - header written
2025-02-12 13:36:40,801 INFO   1448 [scrape_reviews] (scraper.py:387): Pousada Itararé  - Scraping reviews for url: Pousada Itararé
2025-02-12 13:36:40,801 INF

## 1.2. Reunindo os Resultados

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
%autoreload 2

import json
import logging
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
import regex as re
import typer
from dateutils import relativedelta
from unidecode import unidecode

# Removendo aviso de debug
os.environ["PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT"] = "3000"
# Removendo limitação de print de dfs
# pd.set_option("display.max_rows", 500)
# pd.set_option("display.max_columns", 500)
# pd.set_option("display.width", 1000)


In [3]:
# Caminho da pasta contendo os csvs
data_path = "scraper/data/2025/02/16/"
input_file = "scraper/input/hotels_2025_v2.csv"
data_folder = "./data"
Path(data_folder).mkdir(exist_ok=True)

In [4]:
from analysis.src.prep import read_data

df = read_data(input_file, data_path)

read_data...
Lendo arquivo de input
input.shape=(1406, 17)
input.columns=Index(['done', 'name_input', 'n_reviews_input', 'sort_by', 'hl', 'url',
       'google_id', 'n_reviews_max', 'dt_selection', 'stars', 'city', 'sigla',
       'estado', 'capital', 'regiao', 'category', 'subtypes'],
      dtype='object')
Lendo metadados dos hotéis baixados no início da raspagem


100%|██████████| 1369/1369 [00:12<00:00, 107.41it/s]


metadata.shape=(1369, 10)
metadata.columns=Index(['name', 'feature_id', 'retrieval_date_metadata', 'place_name',
       'address', 'overall_rating', 'n_reviews', 'topics', 'url', 'file_name'],
      dtype='object')
Lendo avaliações dos hotéis


100%|██████████| 1372/1372 [00:39<00:00, 34.38it/s]


df_reviews.shape=(792603, 19)
df_reviews.columns=Index(['token', 'review_id', 'retrieval_date', 'rating', 'rating_max',
       'relative_date', 'likes', 'other_ratings', 'trip_type_travel_group',
       'user_name', 'user_is_local_guide', 'user_reviews', 'user_photos',
       'user_url', 'text', 'response_text', 'response_relative_date', 'errors',
       'file_name'],
      dtype='object')
Removendo lixo das avaliações dos hotéis
df_reviews.shape=(791226, 19)
Unindo as diferentes fontes de dados
Merge input metadata
df_hotels.shape=(1406, 26)
Removendo hotéis sem metadados (file_name.isna())
df_hotels.shape=(1369, 26)
Merge avaliações
df_merge.shape=(791123, 44)
df_merge.columns=Index(['done', 'name_input', 'n_reviews_input', 'sort_by', 'hl', 'url',
       'google_id', 'n_reviews_max', 'dt_selection', 'stars', 'city', 'sigla',
       'estado', 'capital', 'regiao', 'category', 'subtypes', 'name',
       'feature_id', 'retrieval_date_metadata', 'place_name', 'address',
       'overall_ra

In [6]:
# Saving
# save_df(df, "data", "df_raw")

In [5]:
# Checking for errors
df.errors.value_counts(dropna=False)

[]    791123
Name: errors, dtype: int64

In [6]:
# Checking for duplicate reviews
duplicate_ids = df[df.review_id.duplicated()].review_id
df_duplicates = df[df.review_id.isin(duplicate_ids)]
print("df_duplicates.shape", df_duplicates.shape)
# Dropping duplicate
df = df.drop_duplicates(subset="review_id").reset_index(drop=True)
print("df.shape", df.shape)

df_duplicates.shape (390, 44)
df.shape (790928, 44)


In [7]:
df.columns

Index(['done', 'name_input', 'n_reviews_input', 'sort_by', 'hl', 'url',
       'google_id', 'n_reviews_max', 'dt_selection', 'stars', 'city', 'sigla',
       'estado', 'capital', 'regiao', 'category', 'subtypes', 'name',
       'feature_id', 'retrieval_date_metadata', 'place_name', 'address',
       'overall_rating', 'n_reviews', 'topics', 'file_name', 'token',
       'review_id', 'retrieval_date', 'rating', 'rating_max', 'relative_date',
       'likes', 'other_ratings', 'trip_type_travel_group', 'user_name',
       'user_is_local_guide', 'user_reviews', 'user_photos', 'user_url',
       'text', 'response_text', 'response_relative_date', 'errors'],
      dtype='object')

In [8]:
df.likes = df.likes.astype(float).replace(-1,0)
df.rating = df.rating.astype(float)
df.user_is_local_guide = df.user_is_local_guide.astype(bool).astype(float)
df.retrieval_date = pd.to_datetime(df.retrieval_date)

In [9]:
from analysis.src.config import agg_dict

# Counting actual number of reviews per hotel
df_count = df.groupby(["name"]).agg(agg_dict).reset_index()
df_count["n_reviews_diff"] = df_count.n_reviews - df_count.review_id
df_count["n_reviews"] = df_count["n_reviews"].astype(int)

In [10]:
df_count[["name","n_reviews_max","n_reviews","review_id","n_reviews_diff"]]

,name,n_reviews_max,n_reviews,review_id,n_reviews_diff
0,50 Hotel,344,363,363,0.0
1,A S Suites,18,23,23,0.0
2,AFPESP - Amparo,1172,1345,1345,0.0
3,AFPESP - Serra Negra,2802,2924,2924,0.0
4,AMORIM POUSADA & POSTO FRECHEIRINHA,70,85,85,0.0
...,...,...,...,...,...
1124,ibis budget Manaus,2776,3404,3404,0.0
1125,pousada morada do sol,36,37,37,0.0
1126,Água Viva Thermas Resort,1555,1601,1601,0.0
1127,Átis Hotel,420,432,432,0.0


In [11]:
# Counting how many reviews have text
df_count["text_percentage"] = df_count["text"] / df_count["review_id"]
df_count[["text_percentage", "text", "review_id"]]
df_count2 = df_count.agg(
    {"text_percentage": "mean", "text": "sum", "review_id": "sum"}
)
df_count2

text_percentage         0.580089
text               461096.000000
review_id          790928.000000
dtype: float64

## 1.3. Tratamento Básico

In [12]:
%timeit
%autoreload 2
from analysis.src.prep import prep_complete
df_prep = prep_complete(df)

df.shape=(790928, 44)
drop duplicates
df.shape=(790928, 44)
likes
df.likes.value_counts()=0.0      729540
1.0       46408
2.0        9329
3.0        2911
4.0        1230
5.0         609
6.0         278
7.0         190
8.0         103
9.0          87
10.0         56
11.0         28
12.0         26
13.0         19
14.0         17
15.0         13
16.0         11
18.0         11
17.0          9
20.0          5
21.0          4
26.0          4
23.0          4
27.0          4
24.0          4
25.0          3
19.0          2
30.0          2
60.0          1
34.0          1
32.0          1
28.0          1
54.0          1
42.0          1
80.0          1
29.0          1
79.0          1
49.0          1
122.0         1
48.0          1
22.0          1
43.0          1
31.0          1
106.0         1
92.0          1
38.0          1
46.0          1
45.0          1
41.0          1
Name: likes, dtype: int64
trip_type_travel_group
df.trip_type_travel_group.value_counts()=[Férias, Família]                   

100%|██████████| 790928/790928 [01:27<00:00, 8996.60it/s] 


Other ratings


100%|██████████| 790928/790928 [01:42<00:00, 7729.07it/s] 


User


100%|██████████| 790928/790928 [00:26<00:00, 30408.58it/s]


In [13]:
df_prep.shape

(790928, 51)

In [14]:
from analysis.src.utils import save_df

save_df(df_prep, "data", "df_prep")

data/df_prep_2025-02-17_09-40-09_370791.pq


## 1.4. Consolidando preps anteriores e Gerando dados para o Índice

In [15]:
path_v1 = "data/df_prep_2024-12-09_08-23-45_627733.pq"
path_v2 = "data/df_prep_2025-02-16_21-16-58_239716.pq"
path_v3 = "data/df_prep_2025-02-17_09-40-09_370791.pq"

In [32]:
df_v2 = pd.read_parquet(path_v2)
print(df_v2.shape)
df_v3 = pd.read_parquet(path_v3)
print(df_v3.shape)

(590361, 51)
(790928, 51)


In [37]:
df_v2["version"] = 2
df_v3["version"] = 3
df_prep = pd.concat([df_v3, df_v2])

In [ ]:
df_prep = df_prep.drop_duplicates("review_id", keep="first")
df_prep.version.value_counts()

3    790928
2       385
Name: version, dtype: int64

In [40]:
df_prep.columns.to_list()

['done',
 'name_input',
 'n_reviews_input',
 'sort_by',
 'hl',
 'url',
 'google_id',
 'n_reviews_max',
 'dt_selection',
 'stars',
 'city',
 'sigla',
 'estado',
 'capital',
 'regiao',
 'category',
 'subtypes',
 'name',
 'feature_id',
 'retrieval_date_metadata',
 'place_name',
 'address',
 'overall_rating',
 'n_reviews',
 'topics',
 'file_name',
 'token',
 'review_id',
 'retrieval_date',
 'rating',
 'rating_max',
 'relative_date',
 'likes',
 'other_ratings',
 'trip_type_travel_group',
 'user_name',
 'user_is_local_guide',
 'user_reviews',
 'user_photos',
 'user_url',
 'text',
 'response_text',
 'response_relative_date',
 'errors',
 'review_date',
 'response_date',
 'rooms',
 'locale',
 'service',
 'highlights',
 'user_id',
 'version']

In [41]:
df_prep.sample().iloc[0]

done                                                                       0
name_input                                                      HOTEL REOBOT
n_reviews_input                                                           -1
sort_by                                                               newest
hl                                                                     pt-br
url                        https://www.google.com/maps/place/HOTEL+REOBOT...
google_id                               0x7070cc204e8fa55:0x363fd3d890f348eb
n_reviews_max                                                            236
dt_selection                                                            2025
stars                                                                    2.0
city                                                               Garanhuns
sigla                                                                     PE
estado                                                            PERNAMBUCO

In [42]:
df_prep[["name_input", "name"]].value_counts()

name_input                                        name                                            
Grand Palladium Imbassaí Resort & Spa             Grand Palladium Imbassaí Resort & Spa               43109
Copacabana Palace                                 Copacabana Palace                                   37414
Costão do Santinho Resort                         Costão do Santinho Resort                           30190
Costa do Sauípe - Premium Sol                     Costa do Sauípe - Premium Sol                       30128
Prive Riviera Park Hotel                          Prive Riviera Park Hotel                            13896
                                                                                                      ...  
Aconchego da Vila                                 Aconchego da Vila                                       8
Vitória da Serra Pousada e Eventos                Vitória da Serra Pousada e Eventos                      8
Casa Azul Campinas - Hospedagem para 

In [43]:
renames = {
    # 'name_input': ,
    # 'n_reviews_input': ,
    # 'sort_by': ,
    # 'hl': ,
    # 'url': ,
    # 'google_id': ,
    # 'n_reviews_max': ,
    # 'dt_selection': ,
    'stars': "estrelas",
    'city': "cidade",
    'sigla': "sigla_estado",
    'estado': "estado",
    'capital': "capital_estado",
    'regiao': "regiao",
    'category': "categoria",
    'subtypes': "subcategoria",
    # 'name': ,
    # 'feature_id': ,
    # 'retrieval_date_metadata': ,
    'place_name': "nome",
    'address': "endereco",
    'overall_rating': "classificacao_geral",
    'n_reviews': "quantidade_avaliacoes",
    # 'topics': ,
    # 'file_name': ,
    # 'token': ,
    # 'review_id': ,
    # 'retrieval_date': ,
    'rating': "nota_avaliacao",
    # 'rating_max': ,
    # 'relative_date': ,
    'likes': "curtidas_avaliacao",
    # 'other_ratings': ,
    # 'trip_type_travel_group': ,
    # 'user_name': ,
    'user_is_local_guide': "usuario_guia_local",
    # 'user_reviews': ,
    # 'user_photos': ,
    # 'user_url': ,
    'text': "text",
    # 'response_text': "texto_resposta",
    # 'response_relative_date': ,
    # 'errors': ,
    'review_date': "data_avaliacao",
    # 'response_date': ,
    'rooms': "nota_quartos",
    'locale': "nota_localizacao",
    'service': "nota_servico",
    # 'highlights': ,
    # 'user_id': ,
    "avaliacao_recente": "avaliacao_recente",
    "text_n_words": "numero_palavras_avaliacao"
}

In [44]:
df_prep["avaliacao_recente"] = (df_prep.review_date > "2024-08-01").astype(float)
df_prep["avaliacao_recente"].value_counts()

0.0    745562
1.0     45751
Name: avaliacao_recente, dtype: int64

In [45]:
df_prep["text_n_words"] = df_prep["text"].fillna("").str.split().apply(lambda x: len(x))

In [46]:
df_prep["response_text"] = df_prep["response_text"].fillna("")

In [47]:
df_prep.review_date = df_prep.review_date.dt.floor("D").dt.strftime('%Y-%m-%d')

In [48]:
df_prep[renames.keys()].sample().iloc[0]

stars                                                                5.0
city                                                                None
sigla                                                                 PR
estado                                                            PARANÁ
capital                                                         CURITIBA
regiao                                                               SUL
category                                                            None
subtypes                                                            None
place_name                                            Wish Foz do Iguaçu
address                Av. das Cataratas, 6845 - Tamandua, Foz do Igu...
overall_rating                                                       4.7
n_reviews                                                         7130.0
rating                                                               5.0
likes                                              

In [49]:
df_index = df_prep.query("text_n_words>=10")[renames.keys()].rename(columns=renames)

In [50]:
df_index.iloc[0].to_dict()

{'estrelas': 0.0,
 'cidade': 'Serra Negra',
 'sigla_estado': 'SP',
 'estado': 'SÃO PAULO',
 'capital_estado': 'SÃO PAULO',
 'regiao': 'SUDESTE',
 'categoria': 'hotéis',
 'subcategoria': 'Hotel',
 'nome': 'Travessa 12 - Suítes',
 'endereco': 'Tv. Ten. Mario Dallari, 12 - Centro, Serra Negra - SP',
 'classificacao_geral': 3.2,
 'quantidade_avaliacoes': 9.0,
 'nota_avaliacao': 5.0,
 'curtidas_avaliacao': 0.0,
 'usuario_guia_local': 1.0,
 'text': 'Excelente custo benefício. Sempre me hospedo lá quando vou pra Serra Negra. Primeira coisa que tem que ficar claro é que é uma hospedagem e não um hotel. O proprietário é muito gente boa e presta assistência dentro do possível. Localização é excelente, na praça dos barzinhos e do lado da Padaria Serrana. Se gosta de agito, é o local perfeito. Se busca silêncio, melhor buscar outra opção mais isolada. Os quartos são grandes. Os preços são bem compatíveis.',
 'data_avaliacao': '2024-07-17',
 'nota_quartos': 4.0,
 'nota_localizacao': 5.0,
 'nota_ser

In [51]:
df_index.shape

(222923, 22)

In [52]:
df_index.to_parquet("data/df_index_2025_v1.pq")